In [ ]:
# scRNA-seq Analysis Pipeline

This notebook performs:
- Cell types annotation
- Differential analysis

## Requirements
- scanpy
- celltypist

In [ ]:
from celltypist import models

# Download the ImmGen model
model = models.Model.load(model='Adult_Mouse_Gut.pkl')

# Annotate
predictions_colon = celltypist.annotate(colon_data, model=model, majority_voting=True)
predictions_spleen = celltypist.annotate(spleen_data, model=model, majority_voting=True)

In [ ]:
colon_data = predictions_colon.to_adata()
spleen_data = predictions_spleen.to_adata()

In [ ]:
colon_data.obs['majority_voting'].value_counts()

In [ ]:
sc.pl.umap(colon_data, color=['batch', 'leiden', 'majority_voting'], size=20)
sc.pl.umap(spleen_data, color=['batch', 'leiden', 'majority_voting'], size=20)

In [ ]:
cd4_cells = colon_data[colon_data.obs['majority_voting'].isin(['Activated CD4+ T cell']), :].copy()

sc.tl.rank_genes_groups(cd4_cells,
                        groupby='batch',
                        method='wilcoxon',  # other options: 't-test', 'logreg'
                        pts=True)  # include fraction of cells expressing the gene

# Store results
de_results = sc.get.rank_genes_groups_df(cd4_cells, group=None)

In [ ]:
# Exclude Rpl, Rps, and mitochondrial genes
de_results_filtered = de_results[~de_results['names'].str.startswith(('Rpl', 'Rps', 'mt-'))]

In [ ]:
de_results_filtered[de_results_filtered['group'] == 'colon_KO'].head(50)

In [ ]:
cd4_cells_spleen = spleen_data[spleen_data.obs['majority_voting'].isin(['Activated CD4+ T cell']), :].copy()

sc.tl.rank_genes_groups(cd4_cells_spleen,
                        groupby='batch',
                        method='wilcoxon',  # other options: 't-test', 'logreg'
                        pts=True)  # include fraction of cells expressing the gene

# Store results
de_results_spleen = sc.get.rank_genes_groups_df(cd4_cells_spleen, group=None)

# Exclude Rpl, Rps, and mitochondrial genes
de_results_spleen_filtered = de_results_spleen[~de_results_spleen['names'].str.startswith(('Rpl', 'Rps', 'mt-'))]

de_results_spleen_filtered[de_results_spleen_filtered['group'] == 'spleen_KO'].head(50)